### Challenge 34: Implement Diffie-Hellman

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection
</div>

<div class="alert alert-block alert-info">

Use the code you just worked out to build a protocol and an "echo" bot. You don't actually have to do the network part of this if you don't want; just simulate that. The protocol is:

```A->B```
Send "p", "g", "A"

```B->A```
Send "B"

```A->B```
Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv

```B->A```
Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv

(In other words, derive an AES key from DH with SHA1, use it in both directions, and do CBC with random IVs appended or prepended to the message).
    
</div>

In [36]:
import numpy as np
import cryptopals as cp
import sha1
from Crypto import Random

Alice creates "A" and sends it to Bob.

```A->B```
Send "p", "g", "A"

In [37]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2

In [38]:
# Alice's private key is 'a'
a = np.random.randint(0, 2**31) % p

# Alice's public key is 'A'
A = pow(g, a, p)

Bob computes his own public key ```B``` using the parameters (```p``` and ```g```) supplied by Alice and sends ```B``` back to her.  I'm not implementing the transfer portion of the protocol...just pretend.

In [39]:
# Bob's private key is 'b'
b = np.random.randint(0, 2**31) % p

# Bob's public key is 'B'
B = pow(g, b, p)

Bob computes a shared secret using Alice's public key ```A```, his private key ```b```, and the public modulus ```p```:

In [40]:
sA = pow(A, b, p)

Now, Bob:
    
1. derives a session key using the shared secret ```sA``` and a random IV.  
2. Encrypts her message using this key
3. Sends Alice the Encrypted message and IV he used.

```AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv```

In [43]:
sA_hex_str = bytes.fromhex(hex(sA)[2:])

key_Bob = sha1.SHA1(sA_hex_str).finish()[0:16]
IV = Random.get_random_bytes(16)

secret_msg = cp.PKCS7_pad(b'What ever you do...don\'t tell Eve!')

ciphertext = cp.AESEncrypt(secret_msg, key_Bob, 'CBC', IV)
data_sent = ciphertext + IV

Alice receive's the encrypted message and IV from Bob and:

1. Derives the session key using the shared secret ```sA``` she computes, along with the IV provided by Bob.
2. Decrypts Bob's message 

In [24]:
# Slight simplication
data_received = data_sent

sB = pow(B, a, p)
sB_hex_str = bytes.fromhex(hex(sB)[2:])
key_Alice = sha1.SHA1(sB_hex_str).finish()[0:16]
IV_received = data_received[-16:]
ciphertext_Alice = data_received[0:-16]
plaintext = cp.AESDecrypt(ciphertext_Alice, key_Alice, 'CBC', IV_received)
                          
print(plaintext.decode())

What ever you do...don't tell Eve!


<div class="alert alert-block alert-info">
    
Now implement the following MITM attack:

```A->M```
Send "p", "g", "A"

```M->B```
Send "p", "g", "p"
    
</div>

In [25]:
# Alice's private key is 'a'
a = np.random.randint(0, 2**31) % p

# Alice's public key is 'A'
A = pow(g, a, p)

Eve sends Bob ```p```, ```g```, ```p``` 
Bob computes a shared secret using ```p``` instead of Alice's real public key, ```A```.

Eve does the same thing to Alice after receiving Bob's public key ```B```.  

<div class="alert alert-block alert-info">
    
```B->M```
Send "B"

```M->A```
Send "p"
</div>

In [46]:
sA_m = pow(p, b, p)
sB_m = pow(p, a, p)

By injecting the public modulus in place of the public key, Eve has forced the shared secret values to be **ZERO**.

<div class="alert alert-block alert-info">

```A->M```
Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
    
```M->B```
Relay that to B

```B->M```
Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv

```M->A```
Relay that to A
    
</div>

Eve obviously has access to both ends.  She could have replaced the public keys with something more believable and decrypted / re-encrypted data going both directions.

Just to prove it works...

In [48]:
sA_m_hex_str = b'\x00'
key_Bob = sha1.SHA1(sA_m_hex_str).finish()[0:16]
IV = Random.get_random_bytes(16)
secret_msg = cp.PKCS7_pad(b'I told you not to tell Eve!')
ciphertext = cp.AESEncrypt(secret_msg, key_Bob, 'CBC', IV)
data_sent = ciphertext + IV

data_received = data_sent
sB_m_hex_str = b'\x00'
key_Alice = sha1.SHA1(sB_m_hex_str).finish()[0:16]
IV_received = data_received[-16:]
ciphertext_Alice = data_received[0:-16]
plaintext = cp.AESDecrypt(ciphertext_Alice, key_Alice, 'CBC', IV_received)
                          
print(plaintext.decode())

I told you not to tell Eve!


<div class="alert alert-block alert-info">
    
M should be able to decrypt the messages. "A" and "B" in the protocol --- the public keys, over the wire --- have been swapped out with "p". Do the DH math on this quickly to see what that does to the predictability of the key.

Decrypt the messages from M's vantage point as they go by.

Note that you don't actually have to inject bogus parameters to make this attack work; you could just generate Ma, MA, Mb, and MB as valid DH parameters to do a generic MITM attack. But do the parameter injection attack; it's going to come up again.

</div>

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)